In [1]:
%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np


In [3]:
import pandas.io.data as web
from pykalman import KalmanFilter
import numpy as np
import pandas as pd

In [4]:
def get_prices(symbol):
    start, end = '2007-05-02', '2016-04-11'
    data = web.DataReader(symbol, 'yahoo', start, end)
    data=pd.DataFrame(data)
    prices=data['Adj Close']
    #prices=np.asarray(list(prices))
    prices=prices.astype(float)
    return prices

In [5]:
gs = get_prices('GS')

In [6]:
def get_results2(prices):
    kf=KalmanFilter(transition_matrices=[[1,1],[0,1]],observation_matrices=[[1,0]],
                    initial_state_mean=[prices[0],0],transition_covariance=[[.001,0],[0,.001]],
                    observation_covariance=1) #TC -> Q ;; OC -> R
    # kf=kf.em(prices,n_iter=5)
    (a,b)=kf.filter(prices)
    test_data = pd.DataFrame(prices)
    test_data.columns = ['prices']
    test_data['signal'] = test_data
    test_data['filter'] = a[:,0]
    test_data['resids'] = test_data.prices - test_data['filter']
    test_data['position'] = 0
    test_data['resid_dev'] = pd.rolling_apply(test_data.resids, 50,  np.std)    # I calculate the threshold on a rolling basis to avoid lookahead bias 
    test_data.position[test_data.resids<-test_data.resid_dev] = 1
    test_data.position[test_data.resids>test_data.resid_dev] = -1
    test_data['daily_diff'] = (test_data.signal.shift(-1) - test_data.signal)/test_data.signal
    test_data['results'] = test_data.position * test_data.daily_diff
    return test_data
    
    

In [7]:
df = get_results2(gs.tolist())

/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#LSTM

In [8]:
sess = tf.InteractiveSession()

In [10]:
sess = tf.Session()

In [11]:
input_ = df.daily_diff
input_ = input_[:-2]
classes = pd.DataFrame(index = df.index)
classes['low'] = df.daily_diff <-.1
classes['l'] = (df.daily_diff> -.1) & (df.daily_diff <0)
classes['h'] = (df.daily_diff> 0) & (df.daily_diff <.1)
classes['high'] = (df.daily_diff> .1) #& (df.daily_diff <.1)

classes = classes*2/2
classes = classes[:-2]
classes = classes[::50]

In [13]:
learning_rate = 0.001
training_iters = 100000
batch_size = len(input_)/50
display_step = 10

In [14]:
n_input = 1 # MNIST data input (img shape: 28*28)
n_steps = 50 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = len(classes.columns) # MNIST total classes (0-9 digits)

In [15]:
x = tf.placeholder("float", [None, n_steps, n_input])
istate = tf.placeholder("float", [None, 2*n_hidden]) #state & cell => 2x n_hidden
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [16]:
def RNN(_X, _istate, _weights, _biases):

    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input]) # (n_steps*batch_size, n_input)
    # Linear activation
    _X = tf.matmul(_X, _weights['hidden']) + _biases['hidden'] #array 2250, 128

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(0, n_steps, _X) # n_steps * (batch_size, n_hidden)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, _X, initial_state=_istate)

    # Linear activation
    # Get inner loop last output
    return tf.matmul(outputs[-1], _weights['out']) + _biases['out']

In [17]:
pred = RNN(x, istate, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [25]:
# Initializing the variables
init = tf.initialize_all_variables()


# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step <10:#* batch_size < training_iters:
        #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        #batch_xs = input_1
        #batch_xs = batch_xs.reshape((batch_size, n_steps, n_input))
        input_1 = input_.reshape((batch_size, n_steps, n_input))
        #print(shape(input_1))
        #print(shape(classes))
        feed_dict={x: input_1, y: classes,
                istate: np.zeros((batch_size, 2*n_hidden))}
        # Fit training using batch data
        sess.run(optimizer, feed_dict=feed_dict)
        step+=1
        '''
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict=feed_dict)
            # Calculate batch loss
            loss = sess.run(cost, feed_dict=feed_dict)
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + \
                  ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")
    # Calculate accuracy for 256 mnist test images
    test_len = 256
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict=feed_dict))
    '''

In [32]:
with tf.Session() as sess:
    
    sess.run(pred, [input_1, np.zeros((batch_size, 2*n_hidden)), weights, biases])
    sess.close()

AttributeError: 'list' object has no attribute 'items'

In [22]:
sess.run(RNN, input_1)

RuntimeError: Attempted to use a closed Session.

In [52]:
weights['hidden']

SyntaxError: invalid syntax (<ipython-input-52-996c572bccca>, line 1)

In [209]:
len(input_[:-2])

2250

In [19]:
df.head()

,prices,signal,filter,resids,position,resid_dev,daily_diff,results
0,196.750037,196.750037,196.750037,0.000000,0,NaN,0.009431,0
1,198.605571,198.605571,197.863654,0.741917,0,NaN,0.026088,0
2,203.786740,203.786740,202.060834,1.725906,0,NaN,-0.000528,-0
3,203.679177,203.679177,204.006166,-0.326989,0,NaN,-0.009990,-0
4,201.644354,201.644354,203.741940,-2.097586,0,NaN,0.007202,0


In [134]:
np.std(df.daily_diff)

0.026411231407140627

In [177]:
input_ = df.daily_diff

In [178]:
input_ = df.daily_diff

classes = pd.DataFrame(index = df.index)
classes['low'] = df.daily_diff <-.1
classes['l'] = (df.daily_diff> -.1) & (df.daily_diff <0)
classes['h'] = (df.daily_diff> 0) & (df.daily_diff <.1)
classes['high'] = (df.daily_diff> .1) #& (df.daily_diff <.1)

classes = classes*2/2

In [7]:
df['smart_choice'] = 0
df['smart_choice'][df.daily_diff>.01] = 1
df['smart_choice'][df.daily_diff<-.01] = -1


/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [94]:
predictors = pd.DataFrame(df.resids)
#predictors = pd.DataFrame()
for i in range(0):
    colname = ''.join(('lag_', str(i)))
    predictors[colname] = df.prices.shift(i)
#predictors = pd.DataFrame(df.resids)
#predictors['lag'] = df.prices.shift(1)
#classes =pd.DataFrame(df.smart_choice)
classes =pd.DataFrame()
classes['long'] = df.smart_choice==1
classes['short'] = df.smart_choice==-1
classes['neutral'] = df.smart_choice==0
classes = classes*2/2


In [95]:
num_predictors = len(predictors.columns )
num_classes = len(classes.columns)

In [96]:
sess = tf.InteractiveSession()

Exception ignored in: <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x10db510f0>>
Traceback (most recent call last):
  File "/Users/liamconnell/anaconda/lib/python3.4/site-packages/tensorflow/python/client/session.py", line 122, in __del__
    self.close()
  File "/Users/liamconnell/anaconda/lib/python3.4/site-packages/tensorflow/python/client/session.py", line 622, in close
    self._default_session.__exit__(None, None, None)
  File "/Users/liamconnell/anaconda/lib/python3.4/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "/Users/liamconnell/anaconda/lib/python3.4/site-packages/tensorflow/python/framework/ops.py", line 2791, in get_controller
    assert self.stack[-1] is default
AssertionError: 


In [97]:
x = tf.placeholder(tf.float32, shape=[None, num_predictors])
y_ = tf.placeholder(tf.float32, shape=[None, num_classes])

In [98]:
weights = tf.Variable(tf.truncated_normal([num_predictors, num_classes], stddev=.001))
bias = tf.Variable(tf.ones([num_classes]))

In [99]:
sess.run(tf.initialize_all_variables())

In [100]:
y = tf.nn.softmax(tf.matmul(x,weights) + bias)

In [101]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [115]:
train_step = tf.train.GradientDescentOptimizer(0.00001).minimize(cross_entropy)

In [116]:
feed_dict  = {
    x: predictors,
    y_: classes
}

In [117]:
for i in range(10000):
  # how to batch:
  #batch = mnist.train.next_batch(50)
  #train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    train_step.run(feed_dict)


In [111]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [106]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [107]:
print(accuracy.eval(feed_dict))

0.464032


In [44]:
y.eval(feed_dict  = {x: predictors})

array([[ 0.27078319,  0.26944414,  0.45977271],
       [ 0.28734705,  0.28805807,  0.42459482],
       [ 0.30851668,  0.31234229,  0.37914106],
       ..., 
       [ 0.15377252,  0.14514266,  0.70108485],
       [ 0.17561856,  0.1675265 ,  0.65685499],
       [ 0.23198934,  0.22695841,  0.54105228]], dtype=float32)

In [35]:
classes =pd.DataFrame()
classes['long'] = df.smart_choice==1
classes['short'] = df.smart_choice==-1
classes['neutral'] = df.smart_choice==0
classes = classes*2/2

In [10]:
classes = classes*2/2

In [10]:
num_predictors = len(predictors.columns )
num_classes = len(classes.columns)

In [11]:
feature_data = tf.placeholder("float",  [None, num_predictors])
actual_classes = tf.placeholder("float",  [None, num_classes])

In [12]:
weights = tf.Variable(tf.truncated_normal([num_predictors, num_classes], stddev=.01))
bias = tf.Variable(tf.ones([num_classes]))

In [13]:
model = tf.nn.softmax(tf.matmul(feature_data, weights) + bias)

In [24]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))


NameError: name 'y_' is not defined

In [14]:
cost = -tf.reduce_sum(actual_classes * tf.log(model))

In [15]:
training_step = tf.train.AdamOptimizer(learning_rate = .001).minimize(cost)

In [16]:
init = tf.initialize_all_variables()

In [17]:
sess.run(init)

In [18]:
for i in range(1000):
    sess.run(
        training_step,
        feed_dict={
            feature_data: predictors,
            actual_classes: classes
        }
    )

In [49]:
predict = tf.argmax(y, 1)

In [20]:
feed_dict  = {
    feature_data: predictors,
    actual_classes: classes
}

In [51]:
zzz = predict.eval(feed_dict, sess)

In [24]:
def tf_confusion_metrics(model, actual_classes, session, feed_dict):
  predictions = tf.argmax(model, 1)
  actuals = tf.argmax(actual_classes, 1)

  ones_like_actuals = tf.ones_like(actuals)
  zeros_like_actuals = tf.zeros_like(actuals)
  ones_like_predictions = tf.ones_like(predictions)
  zeros_like_predictions = tf.zeros_like(predictions)

  tp_op = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
        tf.equal(actuals, ones_like_actuals), 
        tf.equal(predictions, ones_like_predictions)
      ), 
      "float"
    )
  )

  tn_op = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
        tf.equal(actuals, zeros_like_actuals), 
        tf.equal(predictions, zeros_like_predictions)
      ), 
      "float"
    )
  )

  fp_op = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
        tf.equal(actuals, zeros_like_actuals), 
        tf.equal(predictions, ones_like_predictions)
      ), 
      "float"
    )
  )

  fn_op = tf.reduce_sum(
    tf.cast(
      tf.logical_and(
        tf.equal(actuals, ones_like_actuals), 
        tf.equal(predictions, zeros_like_predictions)
      ), 
      "float"
    )
  )

  tp, tn, fp, fn = \
    session.run(
      [tp_op, tn_op, fp_op, fn_op], 
      feed_dict
    )
  try:
    tpr = float(tp)/(float(tp) + float(fn))

    fpr = float(fp)/(float(tp) + float(fn))

    accuracy = (float(tp) + float(tn))/(float(tp) + float(fp) + float(fn) + float(tn))

    recall = tpr
    precision = float(tp)/(float(tp) + float(fp))

    f1_score = (2 * (precision * recall)) / (precision + recall)

    print('Precision = ', precision)
    print('Recall = ', recall)
    print('F1 Score = ', f1_score)
    print('Accuracy = ', accuracy)
  except:
    pass

In [21]:
tf_confusion_metrics(model, actual_classes, sess, feed_dict)

In [7]:
len(training_predictors.)

,0
0,1
1,2
2,3
